In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Charger les données (Exemple : 'sales_data.csv' avec colonnes ['date', 'sales'])
df = pd.read_csv('sales_data.csv', parse_dates=['date'], index_col='date')
df = df.resample('D').sum()  # Assurez-vous que les données sont en base journalière

# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# Création des séquences pour LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 30  # Utiliser 30 jours de données pour prédire le jour suivant
X, y = create_sequences(df_scaled, seq_length)

# Séparation des données en train/test
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# Construction du modèle LSTM
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(50, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Entraînement
epochs = 20
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=16)

# Prédictions
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

# Affichage des résultats
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Ventes réelles')
plt.plot(y_pred, label='Prédictions LSTM', linestyle='dashed')
plt.legend()
plt.title('Prédiction des ventes avec LSTM')
plt.show()
